# 이미지 분류하기
합성곱 신경망을 이해하려면 먼저 합성곱 연산과 교차 상관 연산에 대해 알아야 한다.  
앞으로 합성곱 연산을 합성곱, 교차상관 연산을 교차상관이라고 줄여 부르겠다.  

</br>

## 합성곱 이해하기

두 배열 x와 w가 있다.  
이 중 원소수가 적은 배열 w의 원소 순서를 뒤집는다.  
이 뒤집은 w를 w^r이라고 한다.  
w^r의 첫 부분을 x의 첫 부분과 맞춰 놓고 서로 곱한다.  
예를 들어 아래와 같다고 가정해보자  
```python
 x = [2, 8, 3, 7, 1, 2, 0, 4, 5]
w^r= [3, 5, 1, 2]
```
이때 각 원소를 곱한 뒤 더한 식은 다음과 같다.  
```
2 * 3 + 9 * 5 + 3 * 1 + 7 * 2 = 63
```
즉, 첫 번째 합성곱의 결과는 63이다.  
  
w^r을 오른쪽으로 한 칸 이동해서 같은 방식으로 곱한 뒤 더한다.  
이때 얻은 값은 48이다.  
</br>

같은 방식으로 끝에 도달할 때까지 반복한다.  
이 때, 얻는 값은 차례대로  
63, 48, 49, 28, 21, 20이다.  
이와 같은 합성곱 수식은 x * w와 같이 표기한다.  
기존의 곱하기와 혼동할 수 있음을 주의  
</br>


## 합성곱 구현하기
넘파이로 구현할 수 있다.